# `Setup`

In [183]:
import pandas as pd
import numpy as np
import os

try:
    # Mounting Colab Drive if possible
    from google.colab import drive
    drive.mount('/content/drive')

    # Cloning repo for colab
    try:
        %cd aml_itu/
        !git pull https://github.com/RasKrebs/aml_itu
    except:
        !git clone https://github.com/RasKrebs/aml_itu
        %cd aml_itu/
    os.environ["COLAB"] = "True"

except:
    # Changing directory into aml_itu
    if os.getcwd().split('/')[-1] != 'aml_itu': os.chdir(os.path.abspath('.').split('aml_itu/')[0]+'aml_itu')
    !git pull origin main --ff-only
    os.environ["COLAB"] = "False"

# Utils Import
from utils.helpers import *
from utils.StatefarmPytorchDataset import StateFarmDataset


# Torch
import torch
from torch import nn
import torchvision
import torchvision.transforms as T
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Printing current working directory
print(os.getcwd())

# Setting up device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print (f"GPU is available")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print('MPS device found.')
else:
    print ("No GPU available, using CPU instead")

From https://github.com/RasKrebs/aml_itu
 * branch            main       -> FETCH_HEAD
fatal: Not possible to fast-forward, aborting.
/Users/rasmuskrebs/Documents/School/semester_3/advanced_ml/aml_itu
MPS device found.


### `Config`

In [2]:
# Loading the config file (if content is in workin directory must mean colab is being used)
config = load_config(eval(os.environ["COLAB"]))


# Training Images
train_img = config['dataset']['images']['train']

# Outputting config
config

{'dataset': {'name': 'state-farm-distracted-driver-detection',
  'colab_path': '/content/drive/MyDrive/aml-distracted-drivers-project',
  'data': '../state-farm-distracted-driver-detection/driver_imgs_list.csv',
  'images': {'train': '../state-farm-distracted-driver-detection/imgs/train',
   'test': '../state-farm-distracted-driver-detection/imgs/test'},
  'class_mapping': {'c0': 'safe driving',
   'c1': 'texting - right',
   'c2': 'talking on the phone - right',
   'c3': 'texting - left',
   'c4': 'talking on the phone - left',
   'c5': 'operating the radio',
   'c6': 'drinking',
   'c7': 'reaching behind',
   'c8': 'hair and makeup',
   'c9': 'talking to passenger'}},
 'outputs': {'path': './outputs'},
 'modeling_params': {'batch_size': 32, 'epochs': 100}}

## `TinyVGG`

In [163]:
# IMG Transformations
img_transforms = T.Compose([
    T.Resize(224),
    T.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True)
    ])

# Target Transformations (Removing the c from the target)
target_transform = T.Lambda(lambda y: torch.tensor(int(y.replace('c', ''))))
# target_transform = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(int(y.replace('c', ''))), value=1))

In [164]:
# Creating the dataset
train_data = StateFarmDataset(config, transform=img_transforms, split='train', target_transform=target_transform)
val_data = StateFarmDataset(config, split='test', target_transform=target_transform)

/Users/rasmuskrebs/.local/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [165]:
img, _ = train_data[100]
plt.imshow(img.permute(1, 2, 0))

#### `Model`

In [166]:
class TinyVGG(torch.nn.Module):
    def __init__(self, 
                filters = 32, 
                num_classes = 10,
                kernel_size = (3,3),
                stride = 1,
                in_channels = 3,
                pool_kernel_size = (2,2)):
        """TinyVGG Model"""
        
        super(TinyVGG, self).__init__()
        
        # First Convolution Block
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels = filters,
                      kernel_size=kernel_size,
                      stride=stride),
            nn.ReLU(),
            nn.BatchNorm2d(filters),
            nn.Conv2d(in_channels=filters,
                      out_channels = filters,
                      kernel_size=kernel_size,
                      stride=stride),
            nn.ReLU(),
            nn.BatchNorm2d(filters),
            nn.MaxPool2d(kernel_size = pool_kernel_size))
        
        # Second Convolution Block
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=filters,
                      out_channels = filters,
                      kernel_size=kernel_size,
                      stride=stride),
            nn.ReLU(),
            nn.BatchNorm2d(filters),
            nn.Conv2d(in_channels=filters,
                      out_channels = filters,
                      kernel_size=kernel_size,
                      stride=stride),
            nn.ReLU(),
            nn.BatchNorm2d(filters),
            nn.MaxPool2d(kernel_size = pool_kernel_size))
        
        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=32*53*71,
                      out_features=num_classes))
        

    def forward(self, x):
        # Pass the data through the convolutional blocks
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        
        # Pass through the fully connected layer
        x = self.fc(x)
        
        return x

In [167]:
# Instantiating the model
model = TinyVGG()

model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in

In [180]:
batch_size = config['modeling_params']['batch_size']
epochs = config['modeling_params']['epochs']
seed = 42

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [169]:
x, y = next(iter(train_dataloader))

# Install torchinfo, import if it's available
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

from torchinfo import summary
summary(model, input_size=x.shape)

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 10]                  --
├─Sequential: 1-1                        [32, 32, 110, 147]        --
│    └─Conv2d: 2-1                       [32, 32, 222, 296]        896
│    └─ReLU: 2-2                         [32, 32, 222, 296]        --
│    └─BatchNorm2d: 2-3                  [32, 32, 222, 296]        64
│    └─Conv2d: 2-4                       [32, 32, 220, 294]        9,248
│    └─ReLU: 2-5                         [32, 32, 220, 294]        --
│    └─BatchNorm2d: 2-6                  [32, 32, 220, 294]        64
│    └─MaxPool2d: 2-7                    [32, 32, 110, 147]        --
├─Sequential: 1-2                        [32, 32, 53, 71]          --
│    └─Conv2d: 2-8                       [32, 32, 108, 145]        9,248
│    └─ReLU: 2-9                         [32, 32, 108, 145]        --
│    └─BatchNorm2d: 2-10                 [32, 32, 108, 145]        64
│    └─C

#### `Training Loop`

In [170]:
def train_step(
    model,
    dataloader,
    loss_fn,
    optimizer,
    device):
    """Train step for a single epoch"""
    # Set model to train mode
    model.train()
    
    # Losses and accuracies
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        # Move data to device
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        out = model(X)
        loss = loss_fn(out, y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update train loss and accuracy
        train_loss += loss.item()
        train_acc += (out.argmax(1) == y).type(torch.float).mean().item()
    
    # Return train loss and accuracy
    return train_loss / len(dataloader), train_acc / len(dataloader)


In [171]:
def validation(
    model,
    dataloader,
    loss_fn,
    device):
    """Validation loop"""
    # Set model to eval mode
    model.eval()
    
    # Setup validation loss and accuracy
    val_loss, val_acc = 0, 0
    
    # Disable gradient calculatio
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            
            # Move data to device
            X, y = X.to(device), y.to(device)
            
            # Forward pass and update validation loss
            out = model(X)
            loss = loss_fn(out, y)
            val_loss += loss.item()
            
            # Update validation accuracy
            val_acc += (out.argmax(1) == y).type(torch.float).mean().item()
            
    # Return validation loss and accuracy
    return val_loss / len(dataloader), val_acc / len(dataloader)

In [186]:
def train(
    model,
    train_dataloader,
    validation_dataloader,
    optimizer,
    loss_fn,
    epochs,
    device):
    
    # History
    history = dict(train_loss=[], 
                   train_acc=[], 
                   val_loss=[], 
                   val_acc=[])
    
    # Loop through epochs
    for epoch in range(epochs):
        # Training
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        
        # Validation
        val_loss, val_acc = validation(model, validation_dataloader, loss_fn, device)
        
        # Save train and val loss/acc
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print info
        print(f"Epoch {epoch+1} of {epochs} - Train loss: {train_loss:.5f} - Train acc: {train_acc:.5f} - Val loss: {val_loss:.5f} - Val acc: {val_acc:.5f}")

In [188]:
# Set random seeds
torch.manual_seed(42) 


# Recreate an instance of TinyVGG
model = TinyVGG().to(device)

# Setup loss function and optimizer 
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=1e-3)

# Start the timer

# Train model_0
model_0_results = train(model=model,
                        train_dataloader=train_dataloader,
                        validation_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=epochs,
                        device=device)

/Users/rasmuskrebs/.local/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


KeyboardInterrupt: 